<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/af_single_sequential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title af_single_sequential
#@markdown implementing [@MartinPacesa](https://twitter.com/MartinPacesa/status/1640005299124707328)'s idea
import os
if not os.path.isdir("params"):
  print("installing AlphaFold...")
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  print("downloading AlphaFold params...")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-03-02.tar")
  os.system("tar -xf alphafold_params_2022-03-02.tar -C params")

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np
import tqdm.notebook
TQDM_BAR_FORMAT = '{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]'
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

#@markdown Specify either pdb/pdb_chain or sequence:
pdb = "6D0T" #@param {type:"string"}
pdb_chain = "A" #@param {type:"string"}
sequence = "" #@param {type:"string"}
num_recycles = 0 #@param ["0", "1", "2", "4", "8", "16"] {type:"raw"}
clear_mem()
if sequence == "":
  pdb_filename = get_pdb(pdb)
  af_model = mk_afdesign_model(protocol="fixbb")
  af_model.prep_inputs(pdb_filename=pdb_filename, chain=pdb_chain)
  af_model.set_seq(mode="wildtype")
  mode = "pdb"
else:
  af_model = mk_afdesign_model(protocol="hallucination")
  af_model.prep_inputs(length=len(sequence))
  af_model.set_seq(sequence)
  mode = "free"

import random, string
path = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
while os.path.isdir(path):
  path = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

seq = af_model._params["seq"].copy()
os.makedirs(path, exist_ok=True)
with tqdm.notebook.tqdm(total=af_model._len, bar_format=TQDM_BAR_FORMAT) as pbar:
  for n in range(1,af_model._len+1):
  
    # replace sequence (this is just for visualization)
    af_model._params["seq"] = seq.copy()
    af_model._params["seq"][:,n:] = 0
    
    # mask part of the input features
    af_model._inputs["seq_mask"] = (np.arange(af_model._len) < n).astype(float)
    af_model.predict(verbose=False, hard=False, num_recycles=num_recycles)
    af_model._save_results(verbose=False)
    af_model.save_current_pdb(f"{path}/{n:04d}.pdb")
    pbar.update(1)

In [ ]:
HTML(af_model.animate(dpi=100))

In [ ]:
af_model.plot_pdb()

In [ ]:
#@title Package and download results
#@markdown If you are having issues downloading the result archive, 
#@markdown try disabling your adblocker and run this cell again. 
#@markdown  If that fails click on the little folder icon to the 
#@markdown  left, navigate to file: `name.result.zip`, 
#@markdown  right-click and select \"Download\" 
#@markdown (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).
from google.colab import files
!zip -r {path}.result.zip {path}/*
files.download(f"{path}.result.zip")